<font size="4">

# Наивный (очень) алгоритм сравнения строк

<pre>


1  StrCmpNaive(text, pattern):
2     lp = length(pattern)
3     lt = length(text)
4     matches = <b>new</b> List
5     <b>for</b> i <b>in</b> range 0..lt <b>exclusively</b>  // что здесь не так?
6         <b>for</b> j <b>in</b> range 0..lp <b>exclusively</b>   
7             <b>if</b> text[i+j] != pattern[j]
8                 <b>break</b>
9             <b>if</b> j == lp - 1
10                matches.append(i)
11    return matches


</pre>
> Ассимптотическая сложность в наихудшем случае = $O(?)$

> Какой паттерн (и для какого текста) будет приводить к максимально неэффективной работе алгоритма? (* в случае, если совпавшие строки _не перекрываются_)
    
> Например, поиск `'aaaaaaab'` в `'aaaaaaaaaaaaaaaaaaaaaa'` и тому подобное. Или поиск паттернов из повторяющихся символов, если допускается частичное пересечение разных выравниваний. К примеру, поиск `'aaaa'` в `'aaaaaaaaaaaaaaaaa'`
    
# Мы можем лучше

Мы будем предполагать следующее:
* Паттерн (в большинстве случаев) значительно меньше, чем сам текст
* Алфавит ограничен, и его мы тоже знаем, естественно. Алфавит - множество символов, из которых состоят текст и паттерн

Одна из (относительно) простых идей, которую можно использовать для ускорения поиска:
> Если встреченный в тексте <i>не совпадающий</i> с паттерном символ <i>вообще</i> не встречается в паттерне, можно смело "проматывать" начало паттерна и располагать его сразу за этим символом

<pre>
1  StrCmpLessNaive(text, pattern):
2     lp = Length(pattern)
3     lt = Length(text)
4     matches = <b>new</b> List
5     preprocessed = Preprocess(pattern)
6     
7     shift = 0
8     <b>while</b> shift < lt - lp + 1:
9         j = 0
10        <b>while</b> j < lp <b>and</b> text[shift+j] == pattern[j]:
11            j += 1
12        <b>if</b> j == lp:
13            matches.append(shift)
14            shift += 1
15        <b>else</b>:
16            shift += UsePreprocessed(text[shift+j], shift, preprocessed)
17     <b>return</b> matches
</pre>

<b>Возникают следующие вопросы:</b>
* Как выглядит применение эвристики "перемотки"? 
* Как улучшить эвристику?



<forn size=10>
    
<b>   
`shift = 0`
<pre><font color="red">H</font>ERE IS A SIMPLE EXAMPLE   
<font color="red">E</font>XAMPLE
^
</pre>
    
`j = 0; shift = shift + j + 1`  
`shift = 1`

<pre>H<font color="green">E</font><font color="red">R</font>E IS A SIMPLE EXAMPLE 
 <font color="green">E</font><font color="red">X</font>AMPLE
  ^
</pre> 
`j = 1; shift = shift + j + 1`  
`shift = 3`

<pre>HER<font color="green">E</font><font color="red"> </font>IS A SIMPLE EXAMPLE 
   <font color="green">E</font><font color="red">X</font>AMPLE
    ^
</pre>
`j = 1; shift = shift + j + 1`  
`shift = 5`

</b>
<font>

> И так далее. Пока решение выглядит очень сырым. Как можно _существенно_ улучшить его?


Чтобы ускорить обработку последовательностей еще сильнее, нам придется взглянуть на проблему с другой стороны - буквально. 

Правда, для этого придется немного переписать основной алгоритм.

## Сравнение строк с (правого) конца паттерна

<pre>
1  StrCmpReversed(text, pattern):
2     lp = Length(pattern)
3     lt = Length(text)
4     matches = <b>new</b> List
5     preprocessed = Preprocess(pattern)
6     
7     shift = 0
8     <b>while</b> shift < lt - lp + 1:
9         j = lp - 1
10        <b>while</b> j >= 0 <b>and</b> text[shift+j] == pattern[j]:
11            j -= 1
12        <b>if</b> j <= 0:
13            matches.append(shift)
14            shift += 1
15        <b>else</b>:
16            <b>if</b> text[j+shift] not in preprocessed == 0:
17                shift += (j + 1)
18            <b>else</b>:
19                shift += 1
20     <b>return</b> matches
</pre>


<forn size=10>
<b> 
    
`shift = 0`
<pre>HERE I<font color="red">S</font> A SIMPLE EXAMPLE   
EXAMPL<font color="red">E</font>
      ^
</pre>
    
`j = 6; shift = shift + j + 1`  
`shift = 7`

<pre>HERE IS A SIM<font color="red">P</font>LE EXAMPLE   
       EXAMPL<font color="red">E</font>
             ^
</pre> 
`shift = shift + 1`  
`shift = 8`

<pre>HERE IS A SIMP<font color="red">L</font>E EXAMPLE   
        EXAMPL<font color="red">E</font>
              ^
</pre> 
`shift = shift + 1`  
`shift = 9`

<pre>HERE IS A S<font color="red">I</font><font color="green">MPLE</font> EXAMPLE   
         EX<font color="red">A</font><font color="green">MPLE</font>
           ^
</pre> 
`j = 2; shift = shift + j + 1`  
`shift = 12`

<pre>HERE IS A SIMPLE E<font color="red">X</font>AMPLE   
            EXAMPL<font color="red">E</font><font color="green"></font>
                  ^
</pre>

`shift = shift + 1`  
`shift = 13`

<pre>HERE IS A SIMPLE EX<font color="red">A</font>MPLE   
             EXAMPL<font color="red">E</font><font color="green"></font>
                   ^
</pre>

`shift = ?`  
Можно ли улучшить эвристику "сдвиг на 1" в данном случае?

</b>
<font>

В случае, если символа, встреченного в тексте, нет в паттерне, весь паттерн сдвигается правее этого символа.

А если есть? Предположим, что мы искали бы не весь паттерн, а только его часть - т.е. брали бы _суффикс_ паттерна до встречи с символом `A`: 

> <pre><b>MPLE</b></pre>

тода решение было бы простым - сдвинуть паттерн правее символа. Иначе говоря, сдвигать его до тех пора, пока можно с уверенностью сказать, что "плохого" символа в паттерне нет.

Теперь вспомним, что у паттерна есть вторая часть (то есть просто его начало):

> <pre><b><font color="blue">EXA</font>MPLE</b></pre>

До этой части можно спокойно проматывать. Если мы сделаем так, то окажется, что эвристика сдвигает последний (самый правый) символ `A`  паттерна как раз до совпадения с таким же символом в тексте.

<b>
<pre>HERE IS A SIMPLE EX<font color="red">A</font>MPLE   
                 EX<font color="red">A</font>MPLE<font color="green"></font>
                       ^
</pre>
</b>

> (нам повезло, и поиск закончился, но это не всегда так)

Как расчитать сдвиг? Ведь "плохой" символ может встретиться не только в конце паттерна, но и во время проверки "в середине" паттерна. Препроцессинг должен покрывать все эти случаи. 


# Подготовка символов

Нам потребуется функция `AlphabetIndex(char)`, которая переводит символ в целое чисо: $f: Char \rightarrow Int$. При помощи этой функции будем получать фиксированные числовые индексы для символов из алфавита.


Вторая функция, более важная - это функция, которая подготовит таблицу для работы эвристики "плохого символа".


<pre>
1  BadCharacterTable(pattern):
2      <b>if</b> Length(pattern) == 0: 
3          <b>return new</b> Array[Int] of shape (|Alphabet|, 0)
4      support = <b>new</b> Array[Int] of -1 of length |Alphabet|
5      table = <b>new</b> Array[Int] of shape (|Alphabet|, 1 + Length(pattern))
6      <b>for</b> i <b>in</b> 0..Length(pattern) exclusively:
7          support[AlphabetIndex(pattern[i])] = i
8          <b>for</b> j <b>in</b> 0..Length(pattern) exclusively:
9              table[j][i+1] = support[j]
10     <b>return</b> table
</pre>

Есть и более простой подход, который создает одномерную таблицу, в которой записаны для каждого символа алфавита расстояние от последнего символа до _самого правого_ вхождения в строку. Если символа вообще нет, записывается длина строки: 

<pre>
1 BadCharacterVariant(pattern):
2     lp = Lentgh(pattern);
3     table = <b>new</b> Table of size |Alphabet|
4     <b>for<b> i <b>in</b> 0..|Alphabet| exclusively:
5         table[i] = lp;
6     <b>for<b> i <b>in</b> 0..(|lp| - 1) exclusively:
7         table[pattern[i]] = lp - 1 - i
8     <b>return</b> table
</pre>

\- но она несовместима с дальнейшей версией кода (из-за разных вовзращаемых значений)


## Пример работы функции с небольшим алфавитом

Алфавит: $\{A, C, G, T\}$

Паттерн: $GATTACA$

Таблица:

<pre><b>
        G   A   T   T   A   C   A
A| -1, -1,  1,  1,  1,  4,  4,  6
C| -1, -1, -1, -1, -1, -1,  5,  5
G| -1,  0,  0,  0,  0,  0,  0,  0
T| -1, -1, -1,  2,  3,  3,  3,  3
</b></pre>


Применяя подобную таблицу, можно гарантированно перемотать до "самого правого" относительно сравниваемого в данный момент "плохого" символа. Этот алгоритм является упрощением алгоритма Бойера-Мура и называется **алгоритмом Бойера-Мура-Хорспула**.

Чуть позже сравним его с "полной" версией.

<pre>
1  BoyerMooreHorspool(text, pattern):
2     lp = Length(pattern)
3     lt = Length(text)
4     matches = <b>new</b> List
5     preprocessed = BadCharacterTable(pattern)
6     
7     shift = 0
8     <b>while</b> shift < lt - lp + 1:
9         j = lp - 1
10        <b>while</b> j >= 0 <b>and</b> text[shift+j] == pattern[j]:
11            j -= 1
12        <b>if</b> j <= 0:
13            matches.append(shift)
14            shift += 1
15        <b>else</b>:
16            shift += (j - prepocessed[alphabet_index(text[shift+j])][j])
17            // Альтернативный вариант с "одномерной" таблицей
18            // shift += prepocessed[alphabet_index(text[shift+j])]) - j
19     <b>return</b> matches
</pre>


# Эвристика "хорошего суффикса"

То, что мы только что рассмотрели выше, называется _"эвристикой плохого символа"_.

**Алгоритм Бойера-Мура** использует дополнительную эвристику, называемую эвристикой _"хорошего суффикса"_. При каждом несовпадении этот алгоритм вычисляет _обе_ эвристики, и затем выбирает сдвиг по наилучшей (то есть мсаксимальный сдвиг).


<font size="4">


## Описание эвристики "хорошего суффикса" с примерами

Для начала - как эвристика вообще работает.

Пусть для данного выравнивания паттерна <b>P</b> и текста <b>T</b> подстрока <b>t</b> текста <b>T</b> совпадает с суффиксом <b>P</b>, но левее <b>t</b> происходит несовпадение. 

<pre>
<b>
SAGGTAGACT<font color="green">AAG</font>ACTTACCCAT
     GAAGC<font color="green">AAG</font>
     
t = AAG
</b>
</pre>

Тогда, если существует, найдем самое правое вхождение <b>t'</b> == <b>t</b> в паттерн <b>P</b>, такое, что символ левее <b>P</b> отличается от символа левее <b>t</b> в паттерне <b>P</b>.

<pre>
<b>
SAGGTAGACT<font color="green">AAG</font>ACTTACCCAT
     <font color="blue">G</font><font color="green">AAG</font><font color="blue">C</font><font color="green">AAG</font>
     
t = AAG после C
t' = AAG после G
</b>
</pre>

Сдвинем паттерн <b>P</b> вправо так, чтобы <b>t'</b> из паттерна совпадало с <b>t</b> из текста.

<pre>
<b>
SAGGTAGACT<font color="green">AAG</font>ACTTACCCAT
         G<font color="green">AAG</font>CAAG
</b>
</pre>

Если же <b>t'</b> в паттерне <b>P</b> не существует, до сдвинем левый край <b>P</b> левее <b>t</b> из текста <b>T</b> как можно _меньше_, чтобы _префикс_ <b>P</b> совпадал с суффиксом <b>t</b>, если такой префикс существует.

<pre>
<b>
паттерн изменен на AGAGCAAG с целью демонстрации

SAGGTAGACT<font color="green">A</font><font color="blue">AG</font>ACTTACCCAT
          |<font color="blue">AG</font>AGCAAG
          |||
          |||
          <font color="green">AAG</font>AGCAAG
</b>
</pre>

Если префикса нет, сдвинем паттерн левее подстроки <b>t</b> из текста.

<pre>
<b>
SAGGTAGACT<font color="green">AAG</font>ACTTACCCAT
             TCAGCAAG
</b>
</pre>

### На примере фразы
Как работает эвристика "хорошего суффикса". Синим обозначен суффикс, размещенный в начале паттерна, в том числе его "воображаемая" часть.

<pre>HERE IS A S<font color="red">I</font><font color="green">MPLE</font> EXAMPLE -> HERE IS A SI<font color="green">MPLE</font> EXAMPLE   
      <font color="blue">MPLE</font>X<font color="red">A</font><font color="green">MPLE</font>         ->             <font color="blue">MPLE</font>XAMPLE
           ^
</pre>           

Для сравнения, "плохой символ" в той же ситуации дает меньший сдвиг:

<pre>HERE IS A S<font color="red">I</font><font color="green">MPLE</font> EXAMPLE -> HERE IS A S<font color="blue">I</font>MPLE EXAMPLE
        <font color="blue">I</font>EX<font color="red">A</font><font color="green">MPLE</font>         ->            <font color="blue">I</font>EXAMPLE
</pre>


> В алгоритме Бойера-Мура после вычисления выбирается лучшая эвристика 


    
## Расчет эвристики "хорошего суффикса"

Для того, чтобы воспользвоаться перечисленными выше правилами, нам нужно составить таблицы сдвигов для того, чтобы усрорить процесс сдвига. Таблицы составляются за счет предобработки _только_ паттерна. 

Расчет таблиц для эвристики - сложный процесс, поэтому мы разберем его по частям, "снизу вверх".  
Первая функция, которая потребуется, ищет повторения строки в самой себе по заданным индексам, и возвращает длину совпавшей подстроки:


<pre>
1  MatchLength(string, i, j):
2      <b>if</b> i == j:
3          <b>return</b> Length(string) - i
4       match_length = 0 
5       ls = Length(s)
6       <b>while</b> i < ls <b>and</b> j < ls <b>and</b> string[i] == string[j]
7            match_length += 1
8            i += 1
9            j += 1
10      <b>return</b> match_length
</pre>

Если вернуться к примеру выше, эта функция дала бы примерно такие результаты:

<pre>
MatchLength(<b>GAAGCAAG</b>, 1, 5) == 3
</pre>
         
Т.е. следующие три символа после 1-го и 5-го включительно совпадают:

<pre><b>G<font color="green">AAG</font>C<font color="green">AAG</font></b></pre>


## Какие таблицы понадобятся для работы алгоритма

Для работы алгоритма используются две одномерные таблицы: `L` и `F`. Таблица `F` используется только в случае, если `L[i]` содержит "пустое" значение, или найдено совпадение паттерна и подстроки.

В таблице L содержится следующая информация:

Для всех `i`, `L[i]` содержит наибольшую (самую правую) позицию меньше `n`, такую, что строка `P[i..n]` совпадает с суффиксом `P[1..L[i]]`, и символ, предшествующий этому суффиксу, не равен `P[i-1]` (то есть символу до `P`). Если такой позиции нет, `L[i]` равен 0.


Таблица `F` описывает величину наибольшего суффикса `P[i..n]` для `F[i]`, который также является префиксом `P`. если такого суффикса нет, `P[i] = 0`

> Пока становится только запутанее. Давайте посмотрим на примеры


### Предобработка

Функция FindPrefixes(string) возвращает массив `Z` такой, что `Z[i]` содержит число, равное длине подстроки `S[i]`, котороая при этом является и префиксом `S`. Если такого префикса нет, то `Z[i]` будет 0.

Пример:
<pre>
FindPrefixes("coconut")
--
[7, 0, 2, 0, 0, 0, 0]

FindPrefixes("aaaaaaa")
--
[7, 6, 5, 4, 3, 2, 1]
</pre>

> Домашнее задание - реализовать данную функцию.

Как вы уже могли преположить, данная функция используется для создания таблиц `L` и `H`. 

Однако, поскольку мы ведем сравнение "с конца" паттерна, то и функцию будем применять к развернутой строке (или сама функция пишется так, чтобы искать суффиксы - в принципе, разницы нет)

<pre>

FindPrefixes(Reversed("EXAMPLE"))
--
[7, 0, 0, 0, 0, 0, 1]

FindPrefixes("ELMAXE")
--
[7, 0, 0, 0, 0, 0, 1]


</pre>

### Создание таблицы сдвигов L

По умолчанию используется эта таблица. Что она содержит?

При помощи этой таблицы можно вычислить сдвиг, если не совпал `i-1` элемент, по следующей формуле:

> `shift = Length(pattern) - L[i] - 1`


<pre><b>
GoodSuffixTable("EXAMPLE")
--
  E   X   A   M   P   L   E
[-1, -1, -1, -1, -1, -1,  0]
-- Сдвиг
[ -,  -,  -,  -,  -,  -,  -,  7]
</b></pre>

\- здесь единственный совпадающий суффикс - это последняя `'E'`

<pre><b>
GoodSuffixTable("GAAGCAAG")
--
  G   A   A   G   C   A   A   G
[-1, -1, -1, -1, -1,  3, -1,  0]
-- Сдвиг
[ -,  -,  -,  -,  -,  4,  -,  7]
</b></pre>

`"G"` в центре паттерна игнорируется, так как сдвигаться до суффикса `'G'` перед которым идет `'A'`, когда алгоритм "споткнулся" об `"AG"`, бессмысленно.  Получается, что "сдвиг" нужен до первой `'G'`.

<pre><b>
GoodSuffixTable("GATGCAAG")
--
  G   A   T   G   C   A   A   G
[-1, -1, -1, -1, -1, -1, -1,  3]
-- Сдвиг
[ -,  -,  -,  -,  -,  -,  -,  4]
</b></pre>

А теперь все нормально, потому что в центре перед `'G'` расположено `'T'`, а не `'A'`.


Таблица используется для того, чтобы правильно свидгать паттерн относительно текста так, чтобы на совпавший в прошлый раз суффикс накладывалась самая правая часть, содержащая ту же подстроку:


<pre>
<b>
SAGGTAGACT<font color="green">AAG</font>ACTTACCCAT
     <font color="blue">G</font><font color="green">AAG</font><font color="blue">C</font><font color="green">AAG</font>

После сдвига:

SAGGTAGACT<font color="green">AAG</font>ACTTACCCAT
         G<font color="green">AAG</font>CAAG
</b>
</pre>

Совпадение строки с самой собой не важно, поэтому `L[0] = -1`. Если `L[i]` сожержит `-1` (подходящего суффикса нет), осуществляется переход к следуюущей таблице, "таблице полного сдивга" `H` ("full shift table"). В случае, если произошло несовпадение символов текста и паттерна на `i-1`-м символе, выполняется сдвиг на `Length(pattern) - L[i]`.

`L[i] = k`, где `S[i:]` совпадает с `S[:k]`. Несовпадающие суффиксы равны `-1`. Например, для `EXAMPLE` будет:  


**Псевдокод алгоритма**:
<pre>
GoodSuffixTable(string):
    L = <b>new</b> Array[Int] of -1 of size Length(string)
    Z = FindPrefixes(Reversed(S)) 
    Z = Reverse(Z)
    <b>for</b> j <b>in</b> 0..Length(S)-1 exclusively:
        i = Length(S) - Z[j]
        if i != Length(S):
            L[i] = j
    return L
    
</pre>


### Создание таблицы полного сдвига H

Это таблица частично совпадающих префиксов. Переходим к ней в ситуации, если совпадающего суффикса не существует.

Используется схожим с таблицей `L` образом: если `i-1`-й элемент паттерна не совпал, то паттерн перемещается на `Length(P) - F[i]` вправо.

По сути, это таблица, содержащую размеры следующей вещи: самого длинного суффикса строки `S[i:]` (можно сказать, что это _суффикс суффикса_ паттерна), который одновременно является и префиксом S.

На примере `"EXAMPLE"`:



<pre><b>
FullShiftTable("BONOBO")
--
               'O'
            'B  O'
         'O  B  O'
      'N  O  B  O'
   'O  N  O  B  O'
'N  O  N  O  B  O'
--
[6, 2, 2, 2, 2, 0]
-- Сдвиг
[0, 4, 4, 4, 4, 6]
</b></pre>

\- Другой пример:


<pre><b>
FullShiftTable("EXAMPLE")
--
[7, 1, 1, 1, 1, 1, 1]
-- Сдвиг
[0, 6, 6, 6, 6, 6, 6]
</b></pre>


**Псевдокод функции**

<pre>
FullShiftTable(string):
    F = <b>new</b> Array[Int] of 0 of size Length(string)
    Z = FindPrefixes(string)
    Z = Reversed(Z)
    longest = 0
    <b>for</b> i <b>in</b> Length(Z):
        <b>if</b> Z[i] == i+1:
            longest = max(Z[i], longest)
        <b>else</b>:
            longest
        F[-i-1] = longest
    return F
</pre>


# Алгоритм Бойера-Мура

Теперь мы (надеюсь) готовы к тому, чтобы добавить эвристику "хорошего суффикса" к основному алгоритму.

Добавим новую эвристику в алгоритм Бойера-Мура-Хорспула:


<pre>
BoyerMoore(pattern, text):
    lp = Length(pattern)
    lt = length(text)

    <b>if</b> lp == 0 <b>or</b> lt == 0 <b>or</b> lt < lp:
        <b>return</b> []
        
    matches = <b>new</b> List
    
    bad_character = BadCharacterTable(pattern)
    L = GoodSuffixTable(pattern)
    F = FullShiftTable(pattern) 
    
    previous_k = -1
    shift = 0
    
    <b>while</b> i >= 0 <b>and</b> shift + i > previous_k <b>and</b> pattern[i] = text[shift+i]:
        i -= 1
        
    <b>if</b> i == -1 <b>or</b> shift + i == previous_k:
        matches.append(shift)
    <b>else</b>:
        char_shift = i - bad_character[AlphabetIndex(text[shift + i])][i]
        <b>if</b> i + 1 == lp:
            suffix_shift = 1
        <b>else if</b> L[i+1] == -1:
            suffix_shift = lp - F[i+1]
        <b>else</b>:
            suffix_shift = lp - L[i+1] - 1
        max_shift = Max(char_shift, suffix_shift)
        <b>if</b> max_shift == i+1:
            previous_k = matches[-1] <b>if</b> matches <b>is not</b> empty <b>else</b> -1
        shift += max_shift
        
    <b>return</b> matches
        
    
</pre>


</font>

<font size="4">

# Сложность алгоритма

* O(n) с учетом соблюдения правила Галила.


</font>

<font size="4">

# Домашняя работа

* Реализовать функцию `FindPrefixes`, которая создает таблицу префиксов (примеры работы есть в материале):
  * Функция `FindPrefixes(string)` возвращает массив `Z` такой, что `Z[i]` содержит число, равное длине подстроки `S[i]`, котороая при этом является и префиксом `S`. Если такого префикса нет, то `Z[i]` будет 0.

  * Есть тест-кейсы: файл `preprocess_test_cases.txt`. Традиционно, первая строка - количество записей, первый столбец - строка, далее через пробел - записанные последовательно значения из таблицы префиксов для данной строки. 
* Реализовать алгоритм Бойера-Мура-Хорспула. 
  * Для проверки есть тест-кейсы: `string_matching_test_cases.tsv`. Это .tsv без заголовка, но в первой строчке общее число записей. Первый столбец - текст (содержит пробелы), после первого таба - столбец с паттерном (также может содержать пробелы), в последнем столбце через пробел _все_ вхождения паттерна в текст, в том числе, перекрывающиеся частично. Если вхождений нет, столбец пуст.
* (для тех, кто пишет на C): проверить работу алгоритма Бойера-Мура, реализация на С, [из википедии](https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore_string-search_algorithm#Implementations). Можно на предложенных тест-кесах, можно на своих.


* [Реализация на Go](https://golang.org/src/strings/search.go)

</font>